In [1]:
import cv2
import csv
import numpy as np
import os
import sklearn

In [2]:
def pull_rows_from_log(path):
    rows = []
    with open(path + '/driving_log.csv') as csv_file:
        reader = csv.reader(csv_file)
        next(reader, None)
        for line in reader:
            rows.append(line)
    return rows

In [3]:
# get images locations by using the various csv files as markers
def get_all_images(path):
    raw_dirs = [x[0] for x in os.walk(path)]
    data_dirs = list(filter(lambda x: os.path.isfile(x + '/driving_log.csv'), raw_dirs))
    center = []
    left = []
    right = []
    angle = []
    for directory in data_dirs:
        rows = pull_rows_from_log(directory)
        for row in rows:
            center.append(directory + '/' + row[0].strip())
            left.append(directory + '/' + row[1].strip())
            right.append(directory + '/' + row[2].strip())
            angle.append(float(row[3]))

    return center, left, right, angle

def stack_all_images(center, left, right, angle, correction):
    img_paths = []
    img_paths.extend(center)
    img_paths.extend(left)
    img_paths.extend(right)
    angles = []
    angles.extend(angle)
    angles.extend([x + correction for x in angle])
    angles.extend([x - correction for x in angle])
    return img_paths, angles


In [4]:
def generator(samples, batch_size=32):
    num_samples = len(samples)
    while True: 
        samples = sklearn.utils.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            imgs = []
            angles = []
            for img_path, angle in batch_samples:
                original_img = cv2.imread(img_path)
                img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
                imgs.append(img)
                angles.append(angle)
                # flip for data augmentation
                imgs.append(cv2.flip(img,1))
                angles.append(angle*-1.0)

            inputs = np.array(imgs)
            outputs = np.array(angles)
            yield sklearn.utils.shuffle(inputs, outputs)

In [2]:
from keras.layers.pooling import MaxPooling2D
from keras.layers import Flatten, Dense, Lambda, Convolution2D, Cropping2D
from keras.models import Sequential, Model
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [6]:
def nvidia_drive():
    model = Sequential()
    model.add(Lambda(lambda x: (x/255.0) - 0.5, input_shape=(160,320,3)))
    model.add(Cropping2D(cropping=((50,20), (0,0))))
    model.add(Convolution2D(24,5,5, subsample=(2,2), activation='relu'))
    model.add(Convolution2D(36,5,5, subsample=(2,2), activation='relu'))
    model.add(Convolution2D(48,5,5, subsample=(2,2), activation='relu'))
    model.add(Convolution2D(64,3,3, activation='relu'))
    model.add(Convolution2D(64,3,3, activation='relu'))
    model.add(Flatten())
    model.add(Dense(100))
    model.add(Dense(50))
    model.add(Dense(10))
    model.add(Dense(1))
    return model

In [7]:
center_paths, left_paths, right_paths, angles = get_all_images('data')
image_paths, measurements = stack_all_images(center_paths, 
                                            left_paths,
                                            right_paths,
                                            angles,
                                            0.2)
print('Num Images: {}'.format(len(image_paths)))

Num Images: 37758


In [8]:
all_samples = list(zip(image_paths, angles))
train_samples, val_samples = train_test_split(all_samples, test_size=0.2)

print('Num train: {}'.format(len(train_samples)))
print('Num validation: {}'.format(len(validation_samples)))

Num train: 30206
Num validation: 7552


In [9]:
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

In [12]:
model = nvidia_drive()
model.compile(loss='mse', optimizer='adam')
train_history = model.fit_generator(train_generator,
                                    len(train_samples),
                                    epochs=1,
                                    verbose=1,
                                    validation_data=validation_generator,
                                    validation_steps=len(validation_samples))

model.save('model.h5')


/Users/alex/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), strides=(2, 2), activation="relu")`
  """
/Users/alex/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), strides=(2, 2), activation="relu")`
  
/Users/alex/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), strides=(2, 2), activation="relu")`
  import sys
/Users/alex/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  
/Users/alex/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D

Epoch 1/1
30206/30206 [==============================] - 2648s 88ms/step - loss: 0.0079 - val_loss: 0.0112
